# Tugas Besar TF4012 Pengukuran Berbasis Citra
### Model Evaluation

**Nama**: Febricetta Zahraketzia Sarwono </br>
**NIM**: 13319080

In [1]:
# import library
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np

# load trained model
savedModel=load_model('signdecoderv2.h5')
savedModel.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 2048)              23587712  
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               1049088   
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 2565      
Total params: 24,639,365
Trainable params: 1,051,653
Non-trainable params: 23,587,712
_________________________________________________________________


In [2]:
# image size
batch_size = 128
img_height = 299
img_width = 299

In [3]:
# testing function
def sign_predict(model,frame):
    img = tf.image.resize(frame,[img_height,img_width])
    img = tf.image.flip_left_right(img)
    img_array = tf.keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0) # Create a batch
    predictions = model.predict(img_array)
    score = tf.nn.softmax(predictions[0])
    perc = 100*score
    return predictions, perc

In [4]:
# metrics evaluation
print("Sign Language Decoder (SIBI)")
print("Metrics Evaluation")
print("File \t A \t C \t I \t R \t T")

A_pred = np.zeros(35)
C_pred = np.zeros(35)
I_pred = np.zeros(35)
R_pred = np.zeros(35)
T_pred = np.zeros(35)

# Pengujian
for i in range (35) :
    test_files = ('test/test%d.jpg'%(i+1))

    img = tf.keras.utils.load_img(test_files, target_size=(img_height, img_width))
    img = tf.image.flip_left_right(img)
    img_array = tf.keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0) # Create a batch

    predictions = savedModel.predict(img_array)
    score = tf.nn.softmax(predictions[0])
    perc = 100*score

    print("Test{:d} \t {:.0f}% \t {:.0f}% \t {:.0f}% \t {:.0f}% \t {:.0f}%".format(i+1, perc[0], perc[1], perc[2], perc[3], perc[4]))

    # Cek matrix
    max = np.max(perc)
    if perc[0] == max :
        A_pred[i] = 1
    elif perc[1] == max :
        C_pred[i] = 1
    elif perc[2] == max :
        I_pred[i] = 1
    elif perc[3] == max :
        R_pred[i] = 1
    else:
        T_pred[i] = 1
    
A_true = np.array([1,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1])
C_true = np.array([0,0,0,0,0,0,0,0,1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0])
I_true = np.array([0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0])
R_true = np.array([0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0])
T_true = np.array([0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,1,0,0,0,0,0,0,0,0])

import sklearn
from sklearn.metrics import confusion_matrix
tn_a, fp_a, fn_a, tp_a = confusion_matrix(A_true,A_pred).ravel()
tn_c, fp_c, fn_c, tp_c = confusion_matrix(C_true,C_pred).ravel()
tn_i, fp_i, fn_i, tp_i = confusion_matrix(I_true,I_pred).ravel()
tn_r, fp_r, fn_r, tp_r = confusion_matrix(R_true,R_pred).ravel()
tn_t, fp_t, fn_t, tp_t = confusion_matrix(T_true,T_pred).ravel()

tp = tp_a + tp_c + tp_i + tp_r + tp_t
tn = tn_a + tn_c + tp_i + tp_r + tp_t
fp = fp_a + fp_c + fp_i + fp_r + fp_t
fn = fn_a + fn_c + fn_i + fn_r + fn_t

print("-----")
print("Kinerja \t A \t C \t I \t R \t T \t Total")
print("True Positive \t {:.0f} \t {:.0f} \t {:.0f} \t {:.0f} \t  {:.0f} \t  {:.0f}".format(tp_a, tp_c, tp_i, tp_r, tp_t, tp))
print("True Negative \t {:.0f} \t {:.0f} \t {:.0f} \t {:.0f} \t  {:.0f} \t  {:.0f}".format(tn_a, tn_c, tn_i, tn_r, tn_t, tn))
print("False Positive \t {:.0f} \t {:.0f} \t {:.0f} \t {:.0f} \t  {:.0f} \t  {:.0f}".format(fp_a, fp_c, fp_i, fp_r, fp_t, fp))
print("False Negative \t {:.0f} \t {:.0f} \t {:.0f} \t {:.0f} \t  {:.0f} \t  {:.0f}".format(fn_a, fn_c, fn_i, fn_r, fn_t, fn))
print("-----")

akurasi = (tp + tn) / (tp + tn + fp + fn)
presisi = tp / (tp + fp)
recall = tp / (tp + fn)
f1 = 2/ (1/presisi + 1/recall)

print("Accuracy \t {:.0f}%".format(akurasi*100))
print("Precission \t {:.0f}%".format(presisi*100))
print("Recall \t\t {:.0f}%".format(recall*100))
print("F1-Score \t {:.0f}%".format(f1*100))

Sign Language Decoder (SIBI)
Metrics Evaluation
File 	 A 	 C 	 I 	 R 	 T
Test1 	 39% 	 15% 	 15% 	 15% 	 15%
Test2 	 15% 	 15% 	 16% 	 38% 	 15%
Test3 	 15% 	 15% 	 17% 	 37% 	 15%
Test4 	 15% 	 15% 	 15% 	 15% 	 40%
Test5 	 16% 	 16% 	 19% 	 33% 	 16%
Test6 	 39% 	 15% 	 15% 	 15% 	 15%
Test7 	 16% 	 16% 	 35% 	 18% 	 16%
Test8 	 15% 	 15% 	 15% 	 15% 	 40%
Test9 	 15% 	 40% 	 15% 	 15% 	 15%
Test10 	 16% 	 32% 	 20% 	 16% 	 16%
Test11 	 15% 	 15% 	 15% 	 15% 	 40%
Test12 	 15% 	 40% 	 15% 	 15% 	 15%
Test13 	 39% 	 15% 	 15% 	 15% 	 15%
Test14 	 39% 	 15% 	 15% 	 15% 	 15%
Test15 	 15% 	 40% 	 15% 	 15% 	 15%
Test16 	 16% 	 16% 	 32% 	 20% 	 16%
Test17 	 16% 	 15% 	 17% 	 37% 	 16%
Test18 	 16% 	 16% 	 34% 	 19% 	 16%
Test19 	 16% 	 16% 	 30% 	 22% 	 16%
Test20 	 15% 	 15% 	 16% 	 38% 	 15%
Test21 	 15% 	 15% 	 15% 	 15% 	 39%
Test22 	 40% 	 15% 	 15% 	 15% 	 15%
Test23 	 15% 	 15% 	 15% 	 15% 	 40%
Test24 	 16% 	 16% 	 34% 	 18% 	 16%
Test25 	 15% 	 40% 	 15% 	 15% 	 15%
Test26 	 15